![Coding_Club_Header](../img/coding_club_header_small.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nhs-pycom/coding-club/blob/main/introduction-to-networkx/introduction-to-networkx.ipynb)

In [ ]:
!pip install pyvis

---
---

# Coding Club - NetworkX

This notebook gives a light and practical introduction to `networkx`

- Homepage - https://networkx.org/
- Docs - https://networkx.org/documentation/stable/index.html
- GitHub - https://github.com/networkx/networkx

**"NetworkX is a Python package for the creation, manipulation, and study of the structure, dynamics, and functions of complex networks"**

## Part 1 - An Introduction

Networks (also confusingly called graphs) are a great way to understand how your data is connected.  Networks are built up from `nodes` (sometimes called vertices), and `edges` which connect them.

For the first part of the below, we follow the `Tutorial` section of the `networkx` documentation in part - https://networkx.org/documentation/stable/tutorial.html

In [ ]:
from itertools import count
from IPython.core.display import display, HTML
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
from pyvis.network import Network
import re

print(nx.__version__)

---
## `Graph` — An undirected graphs (with self loops)

https://networkx.org/documentation/stable/reference/classes/graph.html

In [ ]:
# Empty Graph
G = nx.Graph()

print(G.nodes)
print(G.edges)

nx.draw(G)

In [ ]:
# In NetworkX, nodes can be any hashable object 
# e.g., a text string, an image, an XML object, another Graph, a customized node object, etc.

# Add a node
G.add_node(1)

print(G.nodes)
print(G.edges)

nx.draw(G, with_labels=True)

In [ ]:
# Add an edge - note it automatically adds any nodes not already present
G.add_edge(1, 2)

print(G.nodes)
print(G.edges)

nx.draw(G, with_labels=True)

In [ ]:
# Add more than one edge at a time
G.add_edges_from([(2, 3), (1, 3)])

print(G.nodes)
print(G.edges)

nx.draw(G, with_labels=True)

In [ ]:
# See the neighbours of a named node
list(G.neighbors(1))

In [ ]:
# Access the adjacency matrix describing the graph - this tells us about the connections we have
print(nx.adjacency_matrix(G).todense())

### Exercise 1:

Construct your own network using the `Graph` type above and include a `self-loop` (an `edge` connecting a `node` to itself) - how does this change the `adjacency matrix`?

In [ ]:
### Solution Here
#
#

---
## `DiGraph` — A directed graphs (with self loops)

https://networkx.org/documentation/stable/reference/classes/digraph.html

In [ ]:
# Empty DiGraph
DG = nx.DiGraph()

nx.draw(DG)

In [ ]:
# Add a directed edge
DG.add_edge(2, 1)

print(DG.nodes)
print(DG.edges)

nx.draw(DG, with_labels=True)

In [ ]:
# Neighbours are now considered as successors as we have direction
print(list(DG.successors(1)))
print(list(DG.successors(2)))

In [ ]:
# Note - our adjacency matrix is no longer symmetric
print(nx.adjacency_matrix(DG).todense())

In [ ]:
# Convert to a `Graph`
H = nx.Graph(DG)

print(H.nodes)
print(H.edges)

nx.draw(H, with_labels=True)

### Exercise 2:

Can you think of an example where using a directed graph would be particularly important?  

Try to make a simple example of this using a `DiGraph` - make the nodes labels which are more relevent to your scenario.

In [ ]:
### Solution Here
#
#

There are further extensions within `networkx` which can have different types of edges connecting nodes in a parallel fashion:

#### `MultiGraph` — Undirected graphs with self loops and parallel edges

https://networkx.org/documentation/stable/reference/classes/multigraph.html

#### `MultiDiGraph` — Directed graphs with self loops and parallel edges

https://networkx.org/documentation/stable/reference/classes/multidigraph.html#

## `Weights` (and `Attributes`)
We can also attach attributes to `edges` - a fundamental one in `networkx` is an `edge weight` which gives the link an associated value.

As a simple example of why this is useful - we could have a case where our `nodes` represent locations of related services, and the `edge` connecting them has a `weight` associated which is the 'as the crow flies' distance.

In [ ]:
# Empty Graph
WG = nx.Graph()

print(WG.nodes)
print(WG.edges)

nx.draw(WG)

In [ ]:
# Add a weighted edge
WG.add_edge(1, 2, weight=0.5)

print(WG.nodes)
print(WG.edges)

labels = nx.get_edge_attributes(WG, 'weight')
pos=nx.spring_layout(WG)

nx.draw_networkx_edge_labels(
    WG, 
    pos=pos,
    edge_labels=labels,
    verticalalignment='top'
)
nx.draw(WG, pos=pos, with_labels=True)

In [ ]:
WG.add_weighted_edges_from(
    [
        (1, 3, 0.75),
        (2, 3, 0.25)
    ]
)

print(WG.nodes)
print(WG.edges)

labels = nx.get_edge_attributes(WG, 'weight')
pos=nx.spring_layout(WG)

nx.draw_networkx_edge_labels(
    WG, 
    pos=pos,
    edge_labels=labels,
    verticalalignment='top'
)
nx.draw(WG, pos=pos, with_labels=True)

In [ ]:
# Note - our adjacency matrix elements are no longer equal to one
print(nx.adjacency_matrix(WG).todense())

---
## A Quick Aside

### There are lots of special 'built-in' networks can be generated easily as well e.g. `Random Lobster`

*A lobster is a tree that reduces to a caterpillar when pruning all leaf nodes.* 

*A caterpillar is a tree that reduces to a path graph when pruning all leaf nodes; setting `p2` to zero produces a caterpillar.*

https://networkx.org/documentation/stable/reference/generated/networkx.generators.random_graphs.random_lobster.htm

In [ ]:
LG = nx.random_lobster(
        n=3, 
        p1=0.75, 
        p2=0.75, 
        seed=0
    )

nx.draw(LG, pos=nx.spring_layout(LG))

----------
----------

# Part 2 - An Adventure with a `Knowledge Graph`

### What is a Knowledge Graph?

*Knowledge graphs (KGs) organise data from multiple sources, capture information about entities of interest in a given domain or task (like people, places or events), and forge connections between them.*

https://www.turing.ac.uk/research/interest-groups/knowledge-graphs

Here is a healthcare example taken from:

**Learning a Health Knowledge Graph from Electronic Medical Records. Nature Scientific Reports, 2017**

*Maya Rotmensch, Yoni Halpern, Abdulhakim Tlimat, Steven Horng, and David Sontag.*

[Paper](https://www.nature.com/articles/s41598-017-05778-z) | [GitHub](https://github.com/clinicalml/HealthKnowledgeGraph)

- A health KG for 157 diseases and 491 symptoms, learned from patients' data using a Noisy-OR Bayesian Network, as described in the paper.  
- The scores represent the model's relative confidence that an edge exists between a pair of nodes (here linking symptoms to diseases).

Let's see what it looks like!

---
### Prepare the data

In [ ]:
# Load the data into a DataFrame
kg_url = 'https://raw.githubusercontent.com/clinicalml/HealthKnowledgeGraph/master/DerivedKnowledgeGraph_final.csv'

df_kg = pd.read_csv(kg_url)
df_kg.head()

In [ ]:
# Clean up the symptoms and their scores
df_kg['split_symptoms'] = df_kg['Symptoms'].apply(
    lambda elem: elem.split(',')
)
df_kg['split_symptoms'] = df_kg['split_symptoms'].apply(
    lambda elem: [(
        ent.split('(')[0].strip(), 
        float(re.search(r'0.\d{3}', ent)[0])
    ) for ent in elem]
)

df_kg = df_kg.drop('Symptoms', axis=1)
print(df_kg.shape)
df_kg.head()

In [ ]:
# Make a long form DataFrame ready for our network
long_rows = []

for n, row in df_kg.iterrows():
    new_rows = list(zip(
        [
            row['Diseases'] for i in range(len(row['split_symptoms']))
        ],
        [elem[0] for elem in row['split_symptoms']],
        [elem[1] for elem in row['split_symptoms']]
    ))
    
    long_rows.extend(new_rows)
    
df_kg_long = pd.DataFrame(
    long_rows, 
    columns=('disease', 'symptom', 'score')
)

print(df_kg_long.shape)

df_kg_long.head()

In [ ]:
# Check the statistics around the scores
df_kg_long['score'].describe()

In [ ]:
# How many symptoms and diseases are there?
# Are the diseases and symptoms all different?
diseases = set(df_kg_long['disease'].unique())
symptoms = set(df_kg_long['symptom'].unique())

print(f"Number of diseases: {len(diseases)}")
print(f"Number of symptoms: {len(symptoms)}")

print(f"\nIntersection of diseases and symptoms:{len(diseases.intersection(symptoms))}")

In [ ]:
# Let's remove the lower scoring edges to reduce the complexity of the network
score_lower_bound = 0.275

df_most_common = df_kg_long[
    (df_kg_long['score'] >= score_lower_bound)
].reset_index(drop=True)

print(df_most_common.shape)
df_most_common.head()

---
### As a Network...

Start simple, let us just make a `Graph` for now...

In [ ]:
KG = nx.Graph()

These edges could be considered as directed, and the edge label being `is_symptom_of` or `has_symptom` depending on the direction

In [ ]:
KG.add_nodes_from(df_most_common['disease'].unique(), node_type='disease')
KG.add_nodes_from(df_most_common['symptom'].unique(), node_type='symptom')

KG.add_weighted_edges_from(
    list(
        df_most_common[['disease', 'symptom', 'score']].to_records(index=False)
    ), weight='weight',
)

print(len(KG.nodes))
print(len(KG.edges))

In [ ]:
# Let's have a look at our network
fig, ax = plt.subplots(figsize=(12, 12))

groups = set(nx.get_node_attributes(KG, 'node_type').values())
mapping = dict(zip(sorted(groups), count()))
nodes = KG.nodes()
colors = [mapping[KG.nodes[n]['node_type']] for n in nodes]

# Make position for layout
pos = nx.spring_layout(
    KG, 
    iterations=50,
    k=0.3
)

# drawing nodes, edges, and labels separately so we can capture collection for colobar
ec = nx.draw_networkx_edges(
    KG, 
    pos, 
    alpha=0.5
)

nc = nx.draw_networkx_nodes(
    KG, 
    pos, 
    nodelist=nodes, 
    node_color=colors,
    node_size=40, 
    cmap=plt.cm.jet
)

lb = nx.draw_networkx_labels(
    KG, 
    pos,
    verticalalignment='bottom'
)

plt.show()

---
### A *better* way of visualising this network is to make it interactive

Try moving a point around and see what happens!

To do this we use a package called `pyvis` - there are a number of other packages that can do this as well

https://pyvis.readthedocs.io/en/latest/index.html

In [ ]:
nt = Network('600px', width='80%', notebook=True)
nt.from_nx(KG)
nt.toggle_physics(True)
nt.show_buttons(filter_=['physics'])
nt.show('nx.html')

# Required for Colab
display(HTML('nx.html'))

Some settings to be aware of for above:
- `gravitationalConstant` = -2000
- `springConstant` = 0.04
- `springLength` = 95

### Exercise 3

What happens to the complexity of the graph if you change the `score_lower_bound`?

---
## A limited selection of other things you can do in `networkx` with graphs

### Connected Components

We can find out how many connected components our graph has (or disconnected if you prefer to think about it that way)

https://networkx.org/documentation/stable/reference/algorithms/component.html

In [ ]:
len(list(nx.connected_components(KG)))

---
### Node Degree and Degree Centrality

We can understand a `nodes` influence in our network by focusing on the `degrees` of the `nodes` in our network: 

- The `degree` measures how many other `nodes` are connected to a given `node` 

- The `degree centrality` is then the fraction of `nodes` a given `node` is connected to in the graph

https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.degree_centrality.html

In [ ]:
KG.degree('pain')

In [ ]:
KG.degree('headache')

In [ ]:
nx.degree_centrality(KG)

In [ ]:
nx.degree_centrality(KG)['pain']

In [ ]:
nx.degree_centrality(KG)['headache']

---
### Paths

We can investigate paths between `nodes` using `networkx` - here we can see which parts of our KG are well connected, or more isolated from other parts

https://networkx.org/documentation/stable/reference/algorithms/simple_paths.html

https://networkx.org/documentation/stable/reference/algorithms/shortest_paths.html

In [ ]:
len([path for path in nx.all_simple_paths(KG, 'back pain', 'abdominal pain')])

In [ ]:
nx.shortest_path(KG, 'back pain', 'abdominal pain')

In [ ]:
nx.shortest_path_length(KG, 'back pain', 'abdominal pain')

### Final Exercise

Have a look in the `networkx` documentation and find how to include the `scores` (here treated as `weights`) in our `shortest_path_length` calculations.

- How does this change the results from above?
- Try looking at other paths within your network and see what you find
- Try decreasing `score_lower_bound` a little and explore how this changes the paths/degree/connection calculations

In [ ]:
### Solution Here
#
#

---
---
## Some other resources:
There are loads of great things you can do with networks, here are a few things to check out:

- https://github.com/briatte/awesome-network-analysis - lots of other network analysis approaches in a variety of different programming languages
- https://github.com/gboeing/osmnx - great package using `OpenStreetMap` data and `networkx` for analysis
- https://epidemicsonnetworks.readthedocs.io/en/latest/EoN.html - `EoN` a python package looking at epidemics on networks 
- https://distill.pub/2021/gnn-intro/ - a very nice introduction to Graph Neural Networks 